In [1]:
import torch
import random
import numpy as np
import os
import torch.nn as nn
from glob import glob
from tqdm import tqdm
from os.path import join
import os.path as osp
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from util import *

def calculate_metric(y_label, y_pred_label):
    mic_accuracy = accuracy_score(y_label, y_pred_label)
    macro_precision = precision_score(y_label, y_pred_label, average='macro')
    macro_recall = recall_score(y_label, y_pred_label, average='macro')
    macro_f1 = f1_score(y_label, y_pred_label, average='macro')
    return mic_accuracy, macro_precision, macro_recall, macro_f1

def eval_spectrum(data, model_file):
    pkl_file = open(data, 'rb')
    data = pickle.load(pkl_file)
    pkl_file.close()
    X = data["data"]
    y = data["label"]
    with open(model_file, 'rb') as f:
        svclassifier_r = pickle.load(f)
    # print("Loaded.")
    SVM_score = svclassifier_r.score(X, y)
    # print("accuracy by score function: " + str(SVM_score))
    ##### CALCULATE METRIC:
    y_pred = svclassifier_r.predict(X)
    acc, pre, rec, f1 = calculate_metric(y, y_pred)
    # print("accuracy: " + str(acc))
    # print("precision: " + str(pre))
    # print("recall: " + str(rec))
    # print("f1 score: " + str(f1))
    return SVM_score, acc, pre, rec, f1

def extract_test_features(test_dir: str, output_test: str, brightness: float, contrast: float, std=0, missing_value=0, quality=100, type_transform='noise'):
    testset = get_test_path(test_dir=test_dir)
    real_path, fake_path = split_real_fake(train_paths=testset)
    extract_spectrum_features(real_path, fake_path, -1, output_test, brightness=brightness, contrast=contrast, std=std, miss_size=missing_value, quality=quality, type_transform=type_transform)
    return output_test


test_dir_template = "/mnt/disk1/doan/phucnp/Dataset/my_extend_data/extend_data_train/{}/test"

checkpoints = {
    "deepfake": "/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/checkpoint/deepfake/spectrum/c_2.000000",
    "faceswap_2d": "/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/checkpoint/faceswap_2d/spectrum/c_2.000000",
    "3dmm": "/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/checkpoint/3dmm/spectrum/c_2.000000",
    "faceswap_3d": "/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/checkpoint/faceswap_3d/spectrum/c_2.000000",
    "monkey": "/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/checkpoint/monkey/spectrum/c_2.000000",
    "reenact": "/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/checkpoint/reenact/spectrum/c_2.000000",
    "stargan": "/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/checkpoint/stargan/spectrum/c_2.000000",
    "x2face": "/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/checkpoint/x2face/spectrum/c_2.000000"
}

brightness = 1.0
contrasts = [1.0, 0.5, 0.75, 1.5, 2.0]
std= 0.0
missing_value = 0.0
quality=100
type_transform = 'contrast_brightness'

for contrast in contrasts:
    print("\n*********************************************************")
    print("             contrast: ", contrast)
    for technique in ['deepfake', '3dmm', 'faceswap_2d', 'faceswap_3d', 'monkey', 'reenact', 'stargan', 'x2face']:
        checkpoint = checkpoints[technique]
        print("\n\n ==================================== {} CHECKPOINT =========================".format(technique))

        my_break = False
        filename = '/contrast{}.pkl'.format(contrast)
        # filename = '/noise{}.pkl'.format(std)
        # filename = '/missing{}.pkl'.format(missing_value)
        # filename = '/compression{}.pkl'.format(quality)
        output_test = '/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/spectrum/output_features/' + get_datasetname(test_dir_template.format(technique)) + filename
        test_features = extract_test_features(test_dir=test_dir_template.format(technique), output_test=output_test, brightness = brightness, contrast=contrast, std=std, missing_value=missing_value, quality=quality, type_transform=type_transform)        
        
        for pkl_file in sorted(glob(join(checkpoint, '*/*.pkl')), key=lambda pkl: int(pkl.split('/')[-2][-1])):
            if 'model' not in pkl_file:
                continue
                
            fold_id = pkl_file.split('/')[-2][-1]
            ####
            SVM_score, acc, pre, rec, f1 = eval_spectrum(test_features, pkl_file)
            print("            pt file: ", '/'.join(pkl_file.split('/')[-2:]))
            print("    FOLD: {}    -   ".format(fold_id))
            print("            accuracy =   {:.6f}       |  {:6f}".format(SVM_score, acc))
            print("            precision =  {:.6f}  |   recall =    {:.6f}   |  f1 =    {:.6f}".format(pre, rec, f1))

            # if brightness == 1.0:
            #     from_acc = float(pkl_file.split('/')[-2].split('_')[0])
            #     # if abs(round(loss, 4) - round(bestloss, 4)) > 0.001:
            #     #     print("         Error in loss: {}, {}".format(loss, bestloss))
            #     #     my_break = True
            #     if abs(round(from_acc, 4) - round(acc, 4)) > 0.001:
            #         print("         Error in acc: {}, {}".format(from_acc, acc))
            #         my_break = True

            #     if my_break:
            #         break



        if my_break:
            break

/mnt/disk1/anaconda3/envs/phucnp/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



*********************************************************
             contrast:  1.0


 ==================================== deepfake CHECKPOINT =========================


100%|██████████| 5000/5000 [00:18<00:00, 275.22it/s]


            pt file:  0.9876_fold_0/model.pkl
    FOLD: 0    -   
            accuracy =   0.987600       |  0.987600
            precision =  0.987711  |   recall =    0.987590   |  f1 =    0.987599
            pt file:  0.9854_fold_1/model.pkl
    FOLD: 1    -   
            accuracy =   0.985400       |  0.985400
            precision =  0.985555  |   recall =    0.985388   |  f1 =    0.985399
            pt file:  0.9858_fold_2/model.pkl
    FOLD: 2    -   
            accuracy =   0.985800       |  0.985800
            precision =  0.985942  |   recall =    0.985788   |  f1 =    0.985799
            pt file:  0.988_fold_3/model.pkl
    FOLD: 3    -   
            accuracy =   0.988000       |  0.988000
            precision =  0.988111  |   recall =    0.987990   |  f1 =    0.987999
            pt file:  0.987_fold_4/model.pkl
    FOLD: 4    -   
            accuracy =   0.987000       |  0.987000
            precision =  0.987105  |   recall =    0.986990   |  f1 =    0.986999




100%|██████████| 5000/5000 [00:16<00:00, 308.01it/s]


...

            pt file:  0.9513_fold_0/model.pkl
    FOLD: 0    -   
            accuracy =   0.867700       |  0.867700
            precision =  0.893391  |   recall =    0.856647   |  f1 =    0.862089
            pt file:  0.9511_fold_1/model.pkl
    FOLD: 1    -   
            accuracy =   0.865000       |  0.865000
            precision =  0.891724  |   recall =    0.853672   |  f1 =    0.859102
            pt file:  0.9504_fold_2/model.pkl
    FOLD: 2    -   
            accuracy =   0.866300       |  0.866300
            precision =  0.892755  |   recall =    0.855061   |  f1 =    0.860512
            pt file:  0.9516_fold_3/model.pkl
    FOLD: 3    -   
            accuracy =   0.865500       |  0.865500
            precision =  0.892049  |   recall =    0.854220   |  f1 =    0.859654
            pt file:  0.9517_fold_4/model.pkl
    FOLD: 4    -   
            accuracy =   0.869700       |  0.869700
            precision =  0.894793  |   recall =    0.858819   |  f1 =    0.864276


100%|██████████| 5000/5000 [00:33<00:00, 151.09it/s]


            pt file:  0.9092_fold_0/model.pkl
    FOLD: 0    -   
            accuracy =   0.618800       |  0.618800
            precision =  0.760895  |   recall =    0.618800   |  f1 =    0.558714
            pt file:  0.9106_fold_1/model.pkl
    FOLD: 1    -   
            accuracy =   0.616900       |  0.616900
            precision =  0.760620  |   recall =    0.616900   |  f1 =    0.555639
            pt file:  0.9088_fold_2/model.pkl
    FOLD: 2    -   
            accuracy =   0.620000       |  0.620000
            precision =  0.762172  |   recall =    0.620000   |  f1 =    0.560403
            pt file:  0.9118_fold_3/model.pkl
    FOLD: 3    -   
            accuracy =   0.616700       |  0.616700
            precision =  0.761213  |   recall =    0.616700   |  f1 =    0.555177
            pt file:  0.9097_fold_4/model.pkl
    FOLD: 4    -   
            accuracy =   0.617700       |  0.617700
            precision =  0.761366  |   recall =    0.617700   |  f1 =    0.556796


100%|██████████| 5000/5000 [00:19<00:00, 251.16it/s]


            pt file:  0.923_fold_0/model.pkl
    FOLD: 0    -   
            accuracy =   0.881400       |  0.881400
            precision =  0.892477  |   recall =    0.881400   |  f1 =    0.880557
            pt file:  0.9229_fold_1/model.pkl
    FOLD: 1    -   
            accuracy =   0.880500       |  0.880500
            precision =  0.891309  |   recall =    0.880500   |  f1 =    0.879669
            pt file:  0.9232_fold_2/model.pkl
    FOLD: 2    -   
            accuracy =   0.884500       |  0.884500
            precision =  0.894837  |   recall =    0.884500   |  f1 =    0.883739
            pt file:  0.9231_fold_3/model.pkl
    FOLD: 3    -   
            accuracy =   0.881000       |  0.881000
            precision =  0.891690  |   recall =    0.881000   |  f1 =    0.880183
            pt file:  0.9226_fold_4/model.pkl
    FOLD: 4    -   
            accuracy =   0.883200       |  0.883200
            precision =  0.893423  |   recall =    0.883200   |  f1 =    0.882436
